In [3]:
"""
Initialisation
"""

import sqlite3

#Search criteria
NMI_CODE = '4103693793'
DATE_WANTED = '2016-04' #yyyy-mm

db_file = "test_data.db"
tb_metering = 'raw_metering'
tb_holidays = 'holidays'
tb_nmi_info = 'nmi_info'
tb_retailer_timeband = 'retailer_timeband'
target_type = ['Retailer', 'Network'] #holiday types
timeband_type = ['PEAK', 'SHOULDER', 'OFFPEAK']
YR = DATE_WANTED.split('-')[0]
MT = DATE_WANTED.split('-')[1]

#used PARAMS
NMI_State = ''
NMI_Tariff = ''
NMI_Retailer = ''
NMI_NetPro = ''
Re_T_band = []

#time band
timeList = ['Peak1_a', 'Peak1_b', 'Peak2_a', 'Peak2_b', \
               'Shoulder1_a', 'Shoulder1_b', 'Shoulder2_a', 'Shoulder2_b', \
              'Off-Peak1_a', 'Off-Peak1_b', 'Off-Peak2_a', 'Off-Peak2_b', \
              'week_a', 'week_b']

re_timeDict = {} # dictionary for retailer timeband
net_timeDict = {} # dictionary for network timeband

timeDictList = [re_timeDict, net_timeDict] # combine as list

In [27]:
"""
Fetch NMI info
"""


# find State for nmi
NMI_State = fetch_NMI_Info('State', tb_nmi_info, NMI_CODE)[0]

# find tariff for nmi
NMI_Tariff = fetch_NMI_Info('Tariff', tb_nmi_info, NMI_CODE)[0]

# find Retailer for nmi
NMI_Retailer = fetch_NMI_Info('Retailer', tb_nmi_info, NMI_CODE)[0]

# find Network Provider for nmi
NMI_NetPro = fetch_NMI_Info('Network_Provider', tb_nmi_info, NMI_CODE)[0]

# find all the time band for a retailer
query = """
SELECT *
FROM retailer_timeband
WHERE Retailer = ?
AND State = ?
"""
t = (NMI_Retailer,NMI_State)
Re_T_band = [correct_time_stype(x) for x in general_query(query, t, True)[0]]

for i, v in enumerate(timeList):
    re_timeDict[v] = Re_T_band[i+2] # add 2 to shift away Retailer and State Columns

#debug
if False:
    # print all items below
    print NMI_Tariff
    print NMI_Retailer
    print NMI_NetPro
    print NMI_State

if False:
    # print all items for the retailer time band
    for i in Re_T_band:
        print i


In [22]:
"""Retailer Section"""

#Peak Usage
re_peak_usage = Usage_Cal(target_type[0],timeband_type[0])[0]
#Shoulder Usage
re_shoulder_usage = Usage_Cal(target_type[0],timeband_type[1])[0]
#Off peak Usage
re_offPeak_usage = Usage_Tot()[0] - re_peak_usage - re_shoulder_usage



"""Network Section"""
"""
#Peak Usage
re_peak_usage = Usage_Cal(target_type[1],timeband_type[0])[0]
#Shoulder Usage
re_shoulder_usage = Usage_Cal(target_type[1],timeband_type[1])[0]
#Off peak Usage
re_offPeak_usage = Usage_Tot()[0] - re_peak_usage - re_shoulder_usage
"""


print re_peak_usage
print re_shoulder_usage
print re_offPeak_usage

2649.504
5893.788
10048.758


# Functions

In [9]:
# create new table
def create_table():
    pass

# check for all tables in db
def find_all_tables():
    pass

# update table
def update_table():
    pass

# import csv file
def import_csv():
    pass

# export csv file
def export_csv():
    pass

def correct_time_stype(time_string):
    key = ":"
    if time_string == None:
        return None
    else:
        if time_string.find(key) == 1:
            return '0' + time_string
        else:
            return time_string
    

# execute a query and return result in list
def general_query(q_info, t, fetch_all = False):
    conn = sqlite3.connect(db_file)
    c = conn.cursor()
    c.execute(q_info,t)
    if fetch_all:
        data = [x for x in c.fetchall()]
    else:
        data = [x[0] for x in c.fetchall()]
    c.close
    return data

def fetch_NMI_Info(field1, table1, NMI1):
    query = """
        SELECT %s
        FROM %s
        WHERE NMI = ?
        """ % (field1, table1)
    
    t = (NMI1,)
    return general_query(query,t)


# return holiday query
def holiday_query(q_state, q_target, q_mth, q_yr):
    """
    this will return a list of holidays of the given state, network/retailer, months
    """
    query = """
        SELECT Date
        FROM holidays
        WHERE State = ? 
        AND Target = ?
        AND strftime('%m', Date) = ?
        AND strftime('%Y', Date) = ?
        
    """
    t = (q_state, q_target, q_mth, q_yr)

    lst = general_query(query,t)
    return holiday_query_edit(lst) 

def holiday_query_edit(lst):
    result = ""
    prefix = "AND date(Reading_Date) !='"
    potfix = "\'\n"
    for i in lst:
        result += (prefix + i + potfix)
    return result

# return time query
def time_query(q_target, time_type):
    if q_target == target_type[0]:
        #Retailer
        d = timeDictList[0]
    elif q_target == target_type[1]:
        #Network
        d = timeDictList[1]
        
    # PEAK
    if time_type == timeband_type[0]:
        return time_query_edit(d['Peak1_a'], d['Peak1_b'], d['Peak2_a'], d['Peak2_b'])
    #SHOULDER
    elif time_type ==timeband_type[1]:
        return time_query_edit(d['Shoulder1_a'], d['Shoulder1_b'],d['Shoulder2_a'], d['Shoulder2_b'])
    #OFFPEAK
    elif time_type ==timeband_type[2]:
        return time_query_edit(d['Off-Peak1_a'], d['Off-Peak1_b'], d['Off-Peak2_a'], d['Off-Peak2_b'])      
    else:
        return ''

def time_query_edit(i1, i2, i3, i4):
    
    result = ""
    stat_1 = ""
    stat_2 = ""
    if i1 != None and i2 !=None:
        stat_1 = "(Time(Reading_Date) >= time('%s') AND Time(Reading_Date) <time('%s'))" % (i1, i2)
    if i3 != None and i4 !=None:
        stat_2 = "(Time(Reading_Date) >= time('%s') AND Time(Reading_Date) <time('%s'))" % (i3, i4)
    if len(stat_1) and len(stat_2):
        result = "AND(" + stat_1 + "OR" + stat_2 + ")"
    elif len(stat_1):
        result = "AND(" + stat_1 + ")"
    elif len(stat_2):
        result = "AND(" + stat_2 + ")"
    return result

In [21]:
# Usage function
def Usage_Cal(target, timeband):
    q_usage = 'E1'
    # find the condition for holiday and timeband for retailer
    q_holiday = holiday_query(NMI_State, target, MT, YR)
    q_time = time_query(target, timeband)

    query = """
    SELECT sum(Value)
    FROM raw_metering
    WHERE NMI_Suffix = ?
    AND strftime('%Y', Reading_Date) = ?
    AND strftime('%m', Reading_Date) = ?
    AND strftime('%w', Reading_Date) BETWEEN ? AND ?
    """ \
    + q_holiday + q_time

    t=(q_usage, YR, MT, re_timeDict['week_a'],re_timeDict['week_b'])
    return general_query(query, t)

def Usage_Tot():
    #find the total usage
    query = """
    SELECT SUM(Value)
    FROM raw_metering
    WHERE NMI_Suffix = 'E1'
    AND strftime('%Y', Reading_Date) = ?
    AND strftime('%m', Reading_Date) = ?
    """
    t = (YR, MT)
    return general_query(query, t)


    

In [29]:
# update table
conn = sqlite3.connect(db_file)

c = conn.cursor()

# find tariff for nmi
table = 'raw_metering'


query = """
UPDATE 
raw_metering
SET Reading_Date = CASE
WHEN substr(Reading_Date, 13,1) = ':'
THEN substr(Reading_Date, 1, 11) || '0' || substr(Reading_Date, 12, 4)
ELSE Reading_Date
END
"""


#c.execute(query)
print 'Done'


conn.close()

Done


# DEBUG SECTION

In [24]:
# debug 1
if False:
    print time_query(target_type[0], 'PEAK')

In [25]:
# debug 2
if False:
    print holiday_query ('NSW', 'Network', '03', '2016')

In [26]:
# debug 3
if False:
    test_val = "0:12"
    print correct_time_stype(test_val)


# Code for SQL

In [ ]:
# update datetime column
UPDATE raw_metering SET Reading_Date = replace(Reading_Date, '/', '-')

SELECT (substr(Reading_Date, 7, 4) || '-' || substr(Reading_Date, 4, 2) || '-' || substr(Reading_Date, 1, 2) as dd)
FROM raw_metering 

# update date formate
UPDATE 
raw_metering
SET Reading_Date = CASE
WHEN Instr(Reading_Date,'/') =2
THEN substr(Reading_Date, 6, 4) || '-' || substr(Reading_Date, 3, 2) || '-0' || substr(Reading_Date, 1, 1) || substr(Reading_Date, Instr(Reading_Date,' '), 6)
WHEN Instr(Reading_Date,'/') =3 
THEN (substr(Reading_Date, 7, 4) || '-' || substr(Reading_Date, 4, 2) || '-' || substr(Reading_Date, 1, 2)) || substr(Reading_Date, Instr(Reading_Date,' '), 6)
ELSE Reading_Date
END

# update time formate
UPDATE 
raw_metering
SET Reading_Date = CASE
WHEN Instr(Reading_Date, ':') = 13  
THEN substr(Reading_Date, 1, 11) || '0' || substr(Reading_Date, 12, 4)
ELSE Reading_Date
END

#
SELECT SUM(Value)
FROM raw_metering
WHERE NMI_Suffix = 'E1' and Quality_Method = 'A' and strftime('%w', Reading_Date) between '0' and '6'

#peak1_start
SELECT Peak1_Start FROM
retailer_timeband,
(SELECT (nmi_info.State) as state1 FROM nmi_info WHERE NMI = '4103693793') as sub_state,
(SELECT (nmi_info.Retailer) as retail FROM nmi_info WHERE NMI = '4103693793') as sub_retail
WHERE retailer_timeband.Retailer = retail AND retailer_timeband.State = state1